In [1]:
!pip install scapy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.2 MB/s eta 0:00:00


In [2]:
from scapy.all import *
import pandas as pd
import json
import time
import numpy as np


In [3]:
pcap_path = "/content/2025-06-13-traffic-analysis-exercise.pcap"
packets = rdpcap(pcap_path)

print("Total packets loaded:", len(packets))


Total packets loaded: 48877


In [4]:
records = []

for pkt in packets:
    entry = {
        "timestamp": pkt.time,
        "src": pkt[IP].src if IP in pkt else None,
        "dst": pkt[IP].dst if IP in pkt else None,
        "proto": pkt.lastlayer().name,
        "sport": pkt.sport if (TCP in pkt or UDP in pkt) else None,
        "dport": pkt.dport if (TCP in pkt or UDP in pkt) else None,
        "flags": pkt[TCP].flags if TCP in pkt else None,
        "length": len(pkt)
    }
    records.append(entry)

df = pd.DataFrame(records)
df.head()


,timestamp,src,dst,proto,sport,dport,flags,length
0,1749828835.686551,0.0.0.0,255.255.255.255,DHCP options,68.0,67.0,None,344
1,1749828835.687766,10.6.13.3,255.255.255.255,DHCP options,67.0,68.0,None,342
2,1749828835.689030,0.0.0.0,255.255.255.255,DHCP options,68.0,67.0,None,387
3,1749828835.690049,10.6.13.3,255.255.255.255,DHCP options,67.0,68.0,None,342
4,1749828835.712625,None,None,Padding,NaN,NaN,None,60


In [5]:
print("Protocol Counts:")
print(df["proto"].value_counts())

print("\nTop Talkers (Source IPs):")
print(df["src"].value_counts().head())

print("\nTop Destinations (Destination IPs):")
print(df["dst"].value_counts().head())


Protocol Counts:
proto
Raw                                             30287
Padding                                         16088
TCP                                              1659
DNS                                               408
LDAP_SASL_Buffer                                  100
SMB COM Transaction Request                        72
LDAP                                               32
SMB2 IOCTL Request                                 30
SMB2 IOCTL Response                                27
NBNS registration request                          23
CLDAP                                              16
DCE/RPC v5 - Bind                                   9
SMB2 Negotiate Protocol Response                    9
DCE/RPC v5 - Bind Ack                               9
SMB2 CLOSE Request                                  8
SMB2 LOGOFF Request                                 8
SMB2 Error Response                                 8
SMB2 CLOSE Response                                 8
SMB2 

In [6]:
def signature_detection(pkt):
    alerts = []

    # SYN scan behavior
    if TCP in pkt and pkt[TCP].flags == "S":
        alerts.append("SYN scan behavior")

    # Malformed IP header
    if IP in pkt and pkt[IP].ihl < 5:
        alerts.append("Malformed IP header")

    # Suspicious ports
    suspicious_ports = {23, 2323, 3389, 4444}
    if TCP in pkt and pkt[TCP].dport in suspicious_ports:
        alerts.append("Suspicious destination port")

    return alerts


In [7]:
def anomaly_detection(pkt):
    alerts = []

    # Low TTL often indicates scanning or spoofing
    if IP in pkt and pkt[IP].ttl < 10:
        alerts.append("Low TTL anomaly")

    # Very large packet
    if len(pkt) > 1500:
        alerts.append("Oversized packet")

    # Rare protocols
    rare = {"IGMP", "EIGRP", "SCTP"}
    if pkt.lastlayer().name in rare:
        alerts.append("Rare protocol detected")

    return alerts


In [8]:
alerts = []

for pkt in packets:
    sig = signature_detection(pkt)
    ano = anomaly_detection(pkt)

    if sig or ano:
        alerts.append({
            "timestamp": pkt.time,
            "src": pkt[IP].src if IP in pkt else None,
            "dst": pkt[IP].dst if IP in pkt else None,
            "alerts": sig + ano,
            "length": len(pkt)
        })

print("Total alerts:", len(alerts))
pd.DataFrame(alerts).head()


Total alerts: 321


,timestamp,src,dst,alerts,length
0,1749828835.714361,10.6.13.133,224.0.0.252,[Low TTL anomaly],75
1,1749828835.843517,10.6.13.133,10.6.13.3,[SYN scan behavior],66
2,1749828835.844998,10.6.13.3,10.6.13.133,[Oversized packet],1514
3,1749828835.846395,10.6.13.133,10.6.13.3,[SYN scan behavior],66
4,1749828835.847018,10.6.13.133,10.6.13.3,[Oversized packet],1514


In [10]:
alerts_serializable = []
for alert in alerts:
    serializable_alert = alert.copy()
    serializable_alert["timestamp"] = float(serializable_alert["timestamp"])
    alerts_serializable.append(serializable_alert)

with open("alert_log.json", "w") as f:
    json.dump(alerts_serializable, f, indent=4)

from google.colab import files
files.download("alert_log.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
df.to_csv("packet_summary.csv", index=False)
files.download("packet_summary.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
wrpcap("cleaned_output.pcap", packets)
files.download("cleaned_output.pcap")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
print(len(packets))


48877


In [14]:
df.head()


,timestamp,src,dst,proto,sport,dport,flags,length
0,1749828835.686551,0.0.0.0,255.255.255.255,DHCP options,68.0,67.0,None,344
1,1749828835.687766,10.6.13.3,255.255.255.255,DHCP options,67.0,68.0,None,342
2,1749828835.689030,0.0.0.0,255.255.255.255,DHCP options,68.0,67.0,None,387
3,1749828835.690049,10.6.13.3,255.255.255.255,DHCP options,67.0,68.0,None,342
4,1749828835.712625,None,None,Padding,NaN,NaN,None,60


In [15]:
df["proto"].value_counts()


,count
proto,
Raw,30287
Padding,16088
TCP,1659
DNS,408
LDAP_SASL_Buffer,100
SMB COM Transaction Request,72
LDAP,32
SMB2 IOCTL Request,30
SMB2 IOCTL Response,27


In [16]:
pd.DataFrame(alerts).head()


,timestamp,src,dst,alerts,length
0,1749828835.714361,10.6.13.133,224.0.0.252,[Low TTL anomaly],75
1,1749828835.843517,10.6.13.133,10.6.13.3,[SYN scan behavior],66
2,1749828835.844998,10.6.13.3,10.6.13.133,[Oversized packet],1514
3,1749828835.846395,10.6.13.133,10.6.13.3,[SYN scan behavior],66
4,1749828835.847018,10.6.13.133,10.6.13.3,[Oversized packet],1514
